In [1]:
# Data Handlers
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps

# Pytorch
import torch
import torch.nn as nn  # NN; networks (CNN, RNN, losses)
import torch.optim as optim  # Optimizers (Adam, Adadelta, Adagrad)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, IterableDataset  # Dataset manager
from torch.autograd import Variable
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn.functional import one_hot

# Other
import datetime
import os
from tqdm import tqdm
from pathlib import Path
from os import listdir
from fastaniso import anisodiff
from time import time


# Graphics
from matplotlib import pyplot as plt
import seaborn as sns


from skimage.metrics import structural_similarity as ssim
from sklearn.preprocessing import OneHotEncoder

# Additional modules
from dataset_creator import generate_csv
from assistive_funcs import filtering_image, check_ssim, check_gmsd, convert_to_grayscale, get_dataset_name
from csv_dataloader import get_train_test_big_data, get_train_test_small_data
from math import floor

d:\Projects\PythonProjects\_venvs\3.9_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.arange(0, 5)

tensor([0, 1, 2, 3, 4])

In [3]:
one_hot(torch.arange(0, 5), num_classes=10)

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [4]:
# Paths
p_main_data = Path("../data")
p_models = Path("../models")

p_scv_folder = p_main_data / "csv_files" # datasets_path
p_img = p_main_data / "images"

p_noised_imgs = p_main_data / "FC_imgs_with_noise"

p_filtered_images = p_main_data / "FC_filtered_images"

p_gray_images = p_main_data / "gray_images"

In [5]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"{dataset_name = }\n{device = }")
device

device(type='cuda')

In [6]:
# Dataset
win_size = 3
step = 7
create_dataset = 1
if create_dataset:
    generate_csv(win_size=win_size, dump_to_file=5000, step=step, force_create_dataset=1, classification=True)

Borders were added, indexes were created. Passed time = 0.90s
Dataset created.               
Total spent time = 1.79s
Total samples = 73960
Dataset name 'W3_S7_L73960.csv'


In [7]:
win_size = 3
step = 5

dataset_name = get_dataset_name(win_size, step, p_scv_folder, classification=True) #r"W5_S1_L3696640.csv"
dataset_name

'classification\\W3_S5_L146410.csv'

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive/')
# !unzip -q /content/gdrive/MyDrive/NIR/data/FC_data/W5_S5_L146410.zip


In [8]:

class DefaultModel(nn.Module):
    def __init__(self, in_len, out_len) -> None:
        super().__init__()
        self.in_len = in_len
        self.out_len = out_len
        # self.hid_n = 40
        
        self.fcs = nn.Sequential(
            nn.Linear(self.in_len, self.in_len),
            nn.BatchNorm1d(self.in_len),
            nn.ReLU(),
            nn.Linear(self.in_len, self.in_len * 2),
            nn.Dropout(0.5),
            nn.BatchNorm1d(self.in_len * 2),
            nn.ReLU(),
            nn.Linear(self.in_len * 2, self.in_len * 3),
            nn.Dropout(0.5),
            nn.BatchNorm1d(self.in_len * 3),
            nn.ReLU(),
            nn.Linear(self.in_len * 3, self.in_len * 3),
            nn.Dropout(0.5),
            nn.BatchNorm1d(self.in_len * 3),
            nn.ReLU(),
            nn.Linear(self.in_len * 3, self.in_len * 3),
            nn.Dropout(0.5),
            nn.BatchNorm1d(self.in_len * 3),
            nn.ReLU(),
            nn.Linear(self.in_len * 3, self.in_len),
            nn.Dropout(0.5),
            nn.BatchNorm1d(self.in_len),
            nn.ReLU(),
            nn.Linear(self.in_len, self.out_len),
        )

    def forward(self, x):
        x = self.fcs(x)
        return x

In [9]:
# losses = []
# losses_append = losses.append

# valid_losses = []
# valid_losses_append = valid_losses.append

# for epoch in range(num_epoches):
#     model.train()
#     train_loader, test_loader = get_train_test_data(scv_folder=p_scv_folder, dataset_name=dataset_name, batch_size=batch_size, train_size=0.8)
#     for batch_ind, (data, targets) in tqdm(enumerate(train_loader)):
#         # Data on cuda
#         data = data.to(device=device)
#         targets = targets.to(device=device)
        
#         # Forward
#         scores = model(data) # Equal to model.forward(data)
#         loss = criterion(scores, targets)

#         # Backprop
#         loss.backward()

#         # Gradient descent or adam step
#         optimizer.step()
#         optimizer.zero_grad()
        
#         if batch_ind % 3 == 0:
#             losses_append(loss.item())
#     scheduler.step()
#     model.eval()
#     with torch.no_grad():
#         for batch_ind, (data, targets) in tqdm(enumerate(test_loader)):
#             data = data.to(device=device)
#             targets = targets.to(device=device)
            
#             # Forward
#             scores = model(data) # Equal to model.forward(data)
#             loss = criterion(scores, targets)
#             valid_losses_append(loss.item())

In [10]:
class FitModel():
    def __init__(self, model, criterion, optimizer, scheduler, p_scv_folder, dataset_name, batch_size, device, num_epoches) -> None:
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.num_epoches = num_epoches
        self.p_scv_folder = p_scv_folder
        self.dataset_name = dataset_name
        self.batch_size = batch_size
        self.train_losses = []
        self.valid_losses = []
        self.device = device        
        

# loop = tqdm(enumerate(self.val_loader, start=1),
#                     leave=True,
#                     total=len(self.val_loader))

#                 loop.set_description(f"Valid epoch: {current_epoch}")
#                 loop.set_postfix(loss=round(loss.item(), 2))


    def _train(self, current_epoch):
        self.model.train()
        total_loss = 0
        for batch_index, (data, targets) in tqdm(enumerate(self.train_loader, start=1)):
            data = data.to(device=self.device)
            targets = one_hot(targets.int(), num_classes=255).float()
            targets = targets.to(device=self.device)
            scores = self.model(data)
            loss = self.criterion(scores, targets)
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            total_loss += loss.item()
        self.train_losses.append(total_loss / batch_index)
        
    def _valid(self, current_epoch):
        total_loss = 0
        self.model.eval()
        with torch.no_grad():
            for batch_index, (data, targets) in tqdm(enumerate(self.test_loader, start=1)):
                data = data.to(device=self.device)
                targets = one_hot(targets.int(), num_classes=255).float()
                targets = targets.to(device=self.device)
                
                # Forward
                scores = self.model(data) # Equal to model.forward(data)
                loss = self.criterion(scores, targets)
                total_loss += loss.item()
        self.valid_losses.append(total_loss / batch_index)
    
    def fit(self):
        for epoch in range(self.num_epoches):
            self.train_loader, self.test_loader = get_train_test_small_data(scv_folder=self.p_scv_folder, dataset_name=self.dataset_name,
                                                                      batch_size=self.batch_size, train_size=0.8)
            self._train(epoch)
            self._valid(epoch)
            if self.scheduler is not None:
                self.scheduler.step()
                
    def plot_graph(self):
        sns.set()
        fig, (ax_train, ax_test) = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
        fig.suptitle('Loss')

        ax_train.set_title("Train loss")
        ax_test.set_title("Valid loss")

        ax_train.set_ylabel('Loss value')
        ax_test.set_ylabel('Loss value')

        ax_train.set_xlabel("Batch")
        ax_test.set_xlabel("Batch")

        sns.lineplot(data=self.train_losses, ax=ax_train)
        sns.lineplot(data=self.valid_losses, ax=ax_test)

        plt.show()


In [11]:
# Hyperparameters 
learning_rate = 0.001
num_epoches = 3
batch_size = 128

model = DefaultModel(in_len=(win_size ** 2), out_len=1).to(device=device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

In [12]:
fit_model = FitModel(model, criterion, optimizer, scheduler, p_scv_folder, dataset_name, batch_size, device, num_epoches)
fit_model.fit()

0it [00:00, ?it/s]


RuntimeError: one_hot is only applicable to index tensor.

In [ ]:
fit_model.plot_graph()

In [ ]:
def check_without_filtering():
    images_names = listdir(p_noised_imgs)
    for name in images_names:
        check_ssim(p_noised_imgs, p_img, name)
    print()
    for name in images_names:
        check_gmsd(p_noised_imgs, p_img, name)
        
def check_after_filtering():
    images_names = listdir(p_noised_imgs)
    for name in images_names:
        check_ssim(p_filtered_images, p_img, name)
    print()
    for name in images_names:
        check_gmsd(p_filtered_images, p_img, name)
        


In [ ]:
check_after_filtering()

In [ ]:
check_without_filtering()

In [ ]:
# p_img = p_raw_image / list_images[2]
# img_arr = np.array(ImageOps.grayscale(Image.open(p_img)))
# niters = [1, 2, 5, 10, 20]
# kappa = [1, 5, 10, 50, 100]
# for i in niters:
#     for k in kappa:
#         img_filtered = anisodiff(img_arr, niter=i, kappa=k)
#         img_filtered = img_filtered.astype(np.uint8)
#         Image.fromarray(img_filtered).save(f"{p_img}_I{i}_K{k}.jpg")

In [ ]:
counter = 0

for i in range(0, 101):
    if i % 2 == 0:
        if i % 3 == 0:
            counter += 1

counter